In [501]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')
dfBitcoin = pd.read_csv('/content/drive/MyDrive/MCM/BCHAIN-MKPRU.csv')
dfGold = pd.read_csv('/content/drive/MyDrive/MCM/LBMA-GOLD.csv').dropna().reset_index(drop = True)


dfBitcoin.columns = ['Date', 'Price']
dfGold.columns = dfBitcoin.columns


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Essential functions

In [502]:
def addDailyReturn(df):
  df['DailyReturn'] = (df.iloc[:,1] / df.iloc[:,1].shift(1)) - 1

addDailyReturn(dfGold)
addDailyReturn(dfBitcoin)

def plotFeatures(df, colIndex): # dfBitcoin or dfGold
  fig = go.Figure()
  colors = ['blue', 'red', 'green', 'purple']
  j = 0
  for cols in colIndex:
    line = go.Scatter(x = df.Date, y = df.iloc[:,cols], line = dict(color = colors[j], width = 2), name = df.columns[cols])
    j+=1
    fig.add_traces(line)
    #fig.update_layout( height = 600, width = 1000, showlegend = True)
  fig.show()
  from pandas.core import window
def calculateEMA(df, alpha, ):
  df['EMA' + str(alpha) ] = df.iloc[:, 1].ewm(span = alpha, adjust = False).mean()

def calculateSMA(df, span):
   bit = df.iloc[:, 1]
   df['SMA' +str(span)] = bit.rolling(window=span).mean()
   
def labelIntersectionPts(df, maName):
  intersectionPts = []
  for ii in range(len(df)-1):
    if (df.Price[ii] > df[maName][ii] and df.Price[ii+1] < df[maName][ii+1]):
      intersectionPts.append(df.Date[ii])
    elif (df.Price[ii] < df[maName][ii] and df.Price[ii+1] > df[maName][ii+1]):
      intersectionPts.append(df.Date[ii])
  intersection = [1 if x in intersectionPts else 0 for x in df.Date]
  df[maName+'Intersection'] = intersection






In [503]:
calculateSMA(dfBitcoin, 8)

In [504]:
labelIntersectionPts(dfBitcoin, 'SMA8')

In [505]:
dfGold

,Date,Price,DailyReturn
0,9/12/16,1324.60,NaN
1,9/13/16,1323.65,-0.000717
2,9/14/16,1321.75,-0.001435
3,9/15/16,1310.80,-0.008284
4,9/16/16,1308.35,-0.001869
...,...,...,...
1250,9/6/21,1821.60,-0.001152
1251,9/7/21,1802.15,-0.010677
1252,9/8/21,1786.00,-0.008962
1253,9/9/21,1788.25,0.001260


# These are just functions, you guys don't really need to read it. But ask me if you have questions


In [506]:

def calculateEMA(df, alpha, ):
  df['EMA' + str(alpha) ] = df.iloc[:, 1].ewm(span = alpha, adjust = False).mean()
def calculateSMA(df, span):
   bit = df.iloc[:, 1]
   df['SMA' +str(span)] = bit.rolling(window=span).mean()




def labelSellBuyHold(df, maName):
  startDate = ""
  maIntersection = maName+"Intersection"
  labelIntersectionPts(df, maName)
  #initialValue = 1000
  for ii in range(len(df)):
    if df.Price[ii] < df[maName][ii] and df[maIntersection][ii] == 1:
      startDate = df.Date[ii]
      break
  jj = df.Date[df.Date == startDate].index[0]

  labels = df[maIntersection].copy()

  count = 0
  for ii in range(jj, len(df[maIntersection])):
    if labels[ii] == 1:
      if count %2 == 0:
        labels[ii] = 'buy'
      else:
        labels[ii] = 'sell'
      count += 1

    else:
      if count%2 == 0:
        labels[ii] = 'cash'

      else:
        labels[ii] = 'hold'

  df['buySell'] = labels

  for ii in range(len(df.buySell)):
    if df.buySell[ii] == 'sell' and df.buySell[ii + 1] == 'buy' and df.buySell[ii + 2] != 'sell':
      df.buySell[ii] = 'hold'
      df.buySell[ii+1] = 'hold'

    if df.buySell[ii] == 'buy' and df.buySell[ii + 1] == 'sell' and df.buySell[ii + 2] != 'buy':
      df.buySell[ii] = 'cash'
      df.buySell[ii+1] = 'cash' 


def calcReturn(df, value, commR):
  count = 0
  for ii in range(len(df.buySell)):
    if df.buySell[ii] == 'buy': #or df.buySell == 'sell':
      value *= 1 + df.DailyReturn[ii]
      count += 1
    if df.buySell[ii] == 'sell':
      count += 1

    if df.buySell[ii] == 'hold':
      value *= 1 + df.DailyReturn[ii]

  return value * (1 - commR)** count

def findBestSMA(df, commR):
  #df = df.loc[:, 'Date':'DailyReturn']
  whichMAisBest = pd.DataFrame(columns = ['FinalWorth', 'percentReturn'])
  for jj in tqdm (range (1, 150)):
    df = df.loc[:, 'Date':'DailyReturn']
    calculateSMA(df, jj)
    labelIntersectionPts(df, 'SMA' + str(jj))
    try:
      labelSellBuyHold(df, df.columns[3])
      finalValue = calcReturn(df, 1000, commR)
      whichMAisBest.loc[df.columns[3]] = [finalValue, finalValue/1000 - 1]
    except:
     continue

  return whichMAisBest

def findBestEMA(df, commR):
  #df = df.loc[:, 'Date':'DailyReturn']
  whichEMAisBest = pd.DataFrame(columns = ['FinalWorth', 'percentReturn'])
  for jj in tqdm (range (1, 150)):
    df = df.loc[:, 'Date':'DailyReturn']
    calculateEMA(df, jj)
    labelIntersectionPts(df, 'EMA' + str(jj))
    try:
      labelSellBuyHold(df, df.columns[3])
      finalValue = calcReturn(df, 1000, commR)
      whichEMAisBest.loc[df.columns[3]] = [finalValue, finalValue/1000 - 1]
    except:
     continue

  return whichEMAisBest
'''df = dfGold.copy()
jj = 64
maName = 'SMA' + str(jj)

calculateSMA(df, jj)
labelIntersectionPts(df, maName)

labelSellBuyHold(df, df.columns[3])'''

"df = dfGold.copy()\njj = 64\nmaName = 'SMA' + str(jj)\n\ncalculateSMA(df, jj)\nlabelIntersectionPts(df, maName)\n\nlabelSellBuyHold(df, df.columns[3])"

# This section is the good stuff. The graphs are pretty self-explanatory.
# Important new observations:
# Gold Strategy: EMA15 133.4% Return
# Bitc Strategy: SMA7  23667119.92% Return

In [ ]:
def plotGeneral(df, colName):
  fig = go.Figure()
  fig.add_traces(go.Scatter(x = df.index, y = df[colName]))
  fig.show()



In [ ]:

goldSMA = findBestSMA(dfGold, 0.01)
goldSMA

  1%|          | 1/149 [00:00<00:17,  8.68it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.p

,FinalWorth,percentReturn
SMA2,23.299407,-0.976701
SMA3,996.870488,-0.003130
SMA4,1398.615231,0.398615
SMA5,1688.750439,0.688750
SMA6,2070.615109,1.070615
...,...,...
SMA145,1583.200102,0.583200
SMA146,1597.743528,0.597744
SMA147,1591.877730,0.591878
SMA148,1591.877730,0.591878


In [ ]:
goldEMA = findBestEMA(dfGold, 0.01)
plotGeneral(goldEMA, 'percentReturn')
goldEMA

  1%|          | 1/149 [00:00<00:15,  9.72it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.p

,FinalWorth,percentReturn
EMA2,786.443438,-0.213557
EMA3,1271.735909,0.271736
EMA4,1831.418816,0.831419
EMA5,2012.583147,1.012583
EMA6,2193.046329,1.193046
...,...,...
EMA145,1509.596948,0.509597
EMA146,1509.596948,0.509597
EMA147,1528.080351,0.528080
EMA148,1545.306479,0.545306


In [ ]:
bitSMA = findBestSMA(dfBitcoin, 0.02)
plotGeneral(bitSMA, 'percentReturn')
bitSMA

  1%|          | 1/149 [00:00<00:22,  6.51it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.p

,FinalWorth,percentReturn
SMA2,5.800918e+01,-0.941991
SMA3,3.197885e+07,31977.847742
SMA4,5.308304e+04,52.083040
SMA5,1.847959e+08,184794.857998
SMA6,1.857526e+08,185751.590129
...,...,...
SMA145,2.112361e+04,20.123610
SMA146,2.021708e+04,19.217080
SMA147,1.879767e+04,17.797673
SMA148,1.879767e+04,17.797673


In [ ]:
bitEMA = findBestEMA(dfBitcoin, 0.02)
plotGeneral(bitEMA, 'percentReturn')
bitEMA

  1%|          | 1/149 [00:00<00:21,  6.74it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.p

,FinalWorth,percentReturn
EMA2,1.373629e+07,13735.294853
EMA3,9.493785e+07,94936.852736
EMA4,1.765983e+08,176597.253822
EMA5,1.923247e+08,192323.745526
EMA6,2.240320e+08,224031.002089
...,...,...
EMA145,5.313790e+05,530.378954
EMA146,5.337337e+05,532.733700
EMA147,5.337337e+05,532.733700
EMA148,5.337337e+05,532.733700


# Add best strategies to both assets

In [507]:
calculateEMA(dfGold, 15)
labelIntersectionPts(dfGold, 'EMA15')
labelSellBuyHold(dfGold, 'EMA15')

calculateSMA(dfBitcoin, 7)
labelIntersectionPts(dfBitcoin, 'SMA7')
labelSellBuyHold(dfBitcoin, 'SMA7')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning:


A value is tryi

# Test Sensitivity
# Finding: same to previous strategy

In [ ]:

def testSensitivity(df):
  temp = pd.DataFrame(columns = ['FinalValue'])
  for ii in np.linspace(0,1, 100):
    temp.loc[ii] = calcReturn(df, 1000, ii)
  return temp

goldSensitivity = testSensitivity(dfGold)

fig = go.Figure()
fig.add_traces(go.Scatter(x = goldSensitivity.index, y = goldSensitivity.FinalValue))

In [ ]:
bitSensitivity = testSensitivity(dfBitcoin)

fig = go.Figure()
fig.add_traces(go.Scatter(x = bitSensitivity.index, y = bitSensitivity.FinalValue))

# Merge DFs and combine strategies

In [508]:
# First add missing dates to gold

missingDateList = [x for x in dfBitcoin.Date if x not in dfGold.Date.values]
dfMissingDate = pd.DataFrame(columns = dfGold.columns)
for ii in range(len(missingDateList)):
  dfMissingDate.loc[ii] = [missingDateList[ii], np.nan, np.nan, np.nan, np.nan, np.nan]

dfGoldwithMissingDates = pd.concat([dfGold, dfMissingDate], ignore_index = True)
dfGoldwithMissingDates['Date'] = pd.to_datetime(dfGoldwithMissingDates['Date'])
dfGoldwithMissingDates = dfGoldwithMissingDates.sort_values(by = ['Date']).reset_index(drop = True)
dfGoldwithMissingDates

,Date,Price,DailyReturn,EMA15,EMA15Intersection,buySell
0,2016-09-11,NaN,NaN,NaN,NaN,NaN
1,2016-09-12,1324.60,NaN,1324.600000,0,0
2,2016-09-13,1323.65,-0.000717,1324.481250,0,0
3,2016-09-14,1321.75,-0.001435,1324.139844,0,0
4,2016-09-15,1310.80,-0.008284,1322.472363,0,0
...,...,...,...,...,...,...
1821,2021-09-06,1821.60,-0.001152,1802.437370,1,sell
1822,2021-09-07,1802.15,-0.010677,1802.401449,0,cash
1823,2021-09-08,1786.00,-0.008962,1800.351268,0,cash
1824,2021-09-09,1788.25,0.001260,1798.838609,0,cash


In [509]:
mergedDf = pd.DataFrame(columns = ['Date','goldPrice', 'goldR','bitcoinPrice', 'bitcoinR', 'goldIntersection', 'bitcoinIntersection'])
mergedDf['Date'] = dfGoldwithMissingDates.Date
mergedDf['goldPrice'] = dfGoldwithMissingDates.Price
mergedDf['goldR'] = dfGoldwithMissingDates.DailyReturn
mergedDf['bitcoinIntersection'] = dfBitcoin.buySell
mergedDf['goldIntersection'] = dfGoldwithMissingDates.buySell
mergedDf['bitcoinPrice'] = dfBitcoin.Price
mergedDf['bitcoinR'] = dfBitcoin.DailyReturn
mergedDf.goldIntersection = dfGoldFinal.buySell.copy()
mergedDf = mergedDf.replace('buy', 'hold')
mergedDf = mergedDf.replace('sell', 'cash')
mergedDf

,Date,goldPrice,goldR,bitcoinPrice,bitcoinR,goldIntersection,bitcoinIntersection
0,2016-09-11,NaN,NaN,621.65,NaN,-1,0
1,2016-09-12,1324.60,NaN,609.67,-0.019271,0,0
2,2016-09-13,1323.65,-0.000717,610.92,0.002050,0,0
3,2016-09-14,1321.75,-0.001435,608.82,-0.003437,0,0
4,2016-09-15,1310.80,-0.008284,610.38,0.002562,0,0
...,...,...,...,...,...,...,...
1821,2021-09-06,1821.60,-0.001152,51769.06,0.036472,cash,hold
1822,2021-09-07,1802.15,-0.010677,52677.40,0.017546,cash,cash
1823,2021-09-08,1786.00,-0.008962,46809.17,-0.111399,cash,cash
1824,2021-09-09,1788.25,0.001260,46078.38,-0.015612,cash,cash


In [510]:
# When we don't want to buy bitcoin, we buy gold
df = mergedDf
for ii in range(len(df)):

  if df.bitcoinIntersection[ii] == 'hold'  and df.goldIntersection[ii] == 'hold':
    df.goldIntersection[ii] == 'cash'





# Finally, calculate combined return

In [511]:

def getNumTrade(df, colName):
  numTrades = 1
  for ii in range(len(df[colName])-1):
    if df[colName][ii] == 'hold' and df[colName][ii + 1] == 'cash':
      numTrades += 1

    if df[colName][ii] == 'cash' and df[colName][ii + 1] == 'hold':
      numTrades += 1
    
  return numTrades


def tradeOrNot(df, asset):
  colName = asset+'Intersection'
  df[asset+'trade'] = np.nan
  for ii in range(len(df[colName])-1):
    if (df[colName][ii] == 'hold' and df[colName][ii + 1] == 'cash') :
      df[asset+'trade'][ii + 1] = 1
    elif (df[colName][ii] == 'cash' and df[colName][ii + 1] == 'hold'):
      df[asset+'trade'][ii+1] = 1
    else:
      df[asset+'trade'][ii+1] = 0

    

   




def stratReturn(mergedDf):
  goldCommRate = 0.99 ** getNumTrade(mergedDf, 'goldIntersection')
  bitCommRate = 0.98 ** getNumTrade(mergedDf, 'bitcoinIntersection')
  returns = []
  percentR = 1
  
  for ii in range(len(mergedDf)):
    if mergedDf.goldIntersection[ii] == 'hold' and mergedDf.goldR[ii] != np.nan:
      percentR *= 1 + mergedDf.goldR[ii]
     # print( mergedDf.goldR[ii])

    if mergedDf.bitcoinIntersection[ii] == 'hold':
      percentR *= 1 + mergedDf.bitcoinR[ii]

    returns.append(percentR)
  percentR *= goldCommRate * bitCommRate
  return percentR, returns

mergedDf
tradeOrNot(mergedDf, 'gold')
tradeOrNot(mergedDf, 'bitcoin')
getNumTrade(mergedDf, 'goldIntersection')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



106

In [512]:
mergedDf = mergedDf.replace(np.nan, 0)
mergedDf['goldtrade'] = mergedDf['goldtrade'].astype(int)
mergedDf['bitcointrade'] = mergedDf['bitcointrade'].astype(int)
stratReturn(mergedDf)[0]

552385.7713712864

In [513]:
def stratReturn(mergedDf, val, commRs):
  
  v = [val, 0, 0]
  portList = []
  portList.append(v)
  goldCommRate = (1- commRs[0]) #** getNumTrade(mergedDf, 'goldIntersection')
  bitCommRate = (1 - commRs[1]) #** getNumTrade(mergedDf, 'bitcoinIntersection')
  returns = []
  percentR = 1
  percentR *= goldCommRate * bitCommRate
  for ii in tqdm(range(len(mergedDf))):
    if mergedDf.goldIntersection[ii] == 'hold' and mergedDf.goldR[ii] != np.nan:
      percentR *= 1 + mergedDf.goldR[ii]
      v = [0, percentR * val, 0]
    if mergedDf.goldtrade[ii] == 1:
      percentR *= goldCommRate
      
      portList.append(v)
     # print( mergedDf.goldR[ii])

    if mergedDf.bitcoinIntersection[ii] == 'hold':
      percentR *= 1 + mergedDf.bitcoinR[ii]
      v = [0, 0, percentR * val]
    if mergedDf.bitcointrade[ii] == 1:
      percentR *= bitCommRate
      
      portList.append(v)

    returns.append(percentR)
  #percentR *= goldCommRate * bitCommRate
  return percentR, returns, portList

In [514]:
grandReturn = stratReturn(mergedDf, 1000, [0.01, 0.02])

mergedDf['goldtrade'].value_counts()
getNumTrade(mergedDf, 'bitcoinIntersection')
#getNumTrade(mergedDf, 'goldIntersection')
grandReturn[0]

100%|██████████| 1826/1826 [00:00<00:00, 16118.22it/s]


552385.7713712862

In [ ]:
returns = pd.DataFrame(columns = ['Dates', 'Return'])
returns['Dates'] = dfBitcoin.Date
returns['Return'] = grandReturn[1]
returns = returns.set_index('Dates')
#grandReturn[1]
plotGeneral(returns, 'Return')

In [499]:
finalAssetValue = (1 + 569352.4751301626) * 1000
finalAssetValue

569353475.1301626

In [518]:
import plotly.express as px
df = px.data.tips()
fig.add_traces( px.histogram(dfBitcoin, x="DailyReturn"))
fig.show()

df = px.data.tips()
fig = px.histogram(dfGold, x="DailyReturn")
fig.show()

ValueError: ignored

# Sensitivity


In [ ]:
a = []
for ii in tqdm(np.linspace(0, 25, 300)):
  temp = stratReturn(mergedDf, 1000, np.array([0.01, 0.02])* ii)
  a.append(temp[0])

In [ ]:
plt.plot(a)

In [ ]:
np.array([0.01, 0.02]) * 5

# Fit sensitivity to curve

In [ ]:
def sens(a,x, c):
  return a/(x ** c)

from scipy.optimize import curve_fit
popt, pcov = curve_fit(sens, np.linspace(0, 25, 300), a)

/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [ ]:
popt

array([1., 1.])